In [1]:
import os.path as p
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import optuna
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch.utils.data.dataloader import DataLoader
from definitions import *
from model_helper_functions import *
from dataset_helper_functions import *
from bi_lstm import BiLSTM
from bert_embedding_model import BertEmbeddingModel
from DebatesDataset import DebatesDataset


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor(0.8151)
tensor(0.1801)


In [2]:
try:
    IS_MASTER
except: 
    IS_MASTER = False

In [3]:
if not IS_MASTER:
    data, features = {}, {}
    emb_model = 'all-mpnet-base-v2'#'multi-qa-mpnet-base-dot-v1'#'all-MiniLM-L6-v2'
    
    dev_path = p.join(PROC_DATA_DIR_PATH, 'dev')
    features_path = p.join(PROC_DATA_DIR_PATH, 'features')

    data_paths = {
        'dev': [
            p.join(dev_path, 'dev.tsv'),
            # p.join(dev_path, 'dev_spacy.pkl'),
            # p.join(features_path, 'dev_stylometric_features.pkl'),
        ],
        'test': [
            p.join(POLIT_DATA_DIR_PATH, 'test', 'test_combined.tsv'),
            # p.join(PROC_DATA_DIR_PATH, 'test', 'test_spacy.pkl'),
            # p.join(features_path, 'test_stylometric_features.pkl'),

        ],
        'train': [
            p.join(POLIT_DATA_DIR_PATH, 'train', 'train_combined.tsv'),
            # p.join(PROC_DATA_DIR_PATH, 'train', 'train_spacy.pkl'),
            # p.join(features_path, 'train_stylometric_features.pkl')
        ],
        'val': [
            p.join(POLIT_DATA_DIR_PATH, 'val', 'val_combined.tsv'),
            # p.join(PROC_DATA_DIR_PATH, 'val', 'val_spacy.pkl'),
            # p.join(features_path, 'val_stylometric_features.pkl'),
        ],
    }

    for dtype, dpaths in data_paths.items():
        try:
            data[dtype] = pd.read_csv(dpaths[0], sep='\t', index_col=False)
        except Exception as e:
            print(e.args)
            exit()

Remove validation records from train dataset.

In [4]:
dev_df, test_df, train_df, val_df = data.values()
rs = 22

In [6]:
# gpu
train_worthy = train_df.loc[train_df['label'] == 1]
val_worthy = train_worthy.iloc[250:, :]
train_worthy = train_worthy.iloc[:250, :]
train_unworthy = train_df.loc[train_df['label'] == 0].iloc[:len(train_worthy)*2, :]
ratio = len(train_unworthy) / len(train_worthy)

imbalanced_train = train_worthy.append(train_unworthy).reset_index(drop=True)

In [7]:
val_worthy = val_df.loc[val_df['label'] == 1]
val_unworthy = val_df.loc[val_df['label'] == 0].sample(n=len(val_worthy), random_state=rs, ignore_index=True)

balanced_val = val_worthy.append(val_unworthy).sample(frac=1, random_state=rs, ignore_index=True)

test_worthy = test_df.loc[test_df['label'] == 1]
test_unworthy = test_df.loc[test_df['label'] == 0].sample(n=len(test_worthy), random_state=rs, ignore_index=True)

balanced_test = test_worthy.append(test_unworthy).sample(frac=1, random_state=rs, ignore_index=True)

Wrap data in `DebatesDataset` class so that it can be passed to `DataLoader` class.

In [12]:
batch_size = 16
transform = None

dd_train = DebatesDataset(data=dev_df, transform=transform)
dd_val = DebatesDataset(data=balanced_val, transform=transform)
dd_test = DebatesDataset(data=balanced_test, transform=transform)

sent_level_feature_dim = len(dd_train[0][3]) if not transform else len(dd_train[0][4])

loader_train = DataLoader(dd_train, batch_size=batch_size, shuffle=True, drop_last=True)
loader_val = DataLoader(dd_val, batch_size=batch_size, shuffle=True, drop_last=True)
loader_test = DataLoader(dd_test, batch_size=batch_size, shuffle=True, drop_last=True)

Initialize model, optimizer and criterion.

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_model = BertEmbeddingModel(device=device)
model = BiLSTM(
    embedding_dim=embedding_model.dim,
    sent_level_feature_dim=sent_level_feature_dim
).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()
n_epochs = 5
eval_period = len(loader_train) // 2
best_val_loss = float("Inf")
exp_path = p.join(EXP_DIR_PATH, 'bi-lstm', 'test-bcewlogitsloss')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialize running values.

In [14]:
running_loss, val_running_loss = 0.0, 0.0
global_step = 0
train_losses, val_losses, global_steps = [], [], []

Training process.

In [ ]:
model.train()

for epoch in range(n_epochs):
    for ids, sentences, labels, features in loader_train:
        labels = labels.float().to(device)
        embeddings, lengths = embedding_model(sentences)

        output, _ = model(embeddings, lengths)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update running values
        running_loss += loss.item()
        global_step += 1

        if global_step % eval_period == 0:
            model.eval()
            with torch.no_grad():
                for val_ids, val_sentences, val_labels, val_features in loader_val:
                    val_labels = val_labels.float().to(device)
                    val_embeddings, val_lengths = embedding_model(sentences)

                    output, _ = model(val_embeddings, val_lengths)

                    loss = criterion(output, val_labels)
                    val_running_loss += loss.item()

            train_losses.append(running_loss / eval_period)
            val_losses.append(val_running_loss / len(loader_val))
            global_steps.append(global_step)

            running_loss, val_running_loss = 0.0, 0.0

            print(
                f'Epoch [{epoch+1}/{n_epochs}], '
                f'Step [{global_step}/{n_epochs*len(loader_train)}], '
                f'Train Loss: {train_losses[-1]:.4f}, '
                f'Validation Loss: {val_losses[-1]:.4f}'
            )

            # TODO: early stopping here ?
            if val_losses[-1] < best_val_loss:
                best_val_loss = val_losses[-1]
                save_checkpoint(
                    p.join(exp_path, 'model.pt'),
                    model=model,
                    optimizer=optimizer,
                    val_loss=best_val_loss
                )
                save_metrics(
                    p.join(exp_path, 'metrics.pt'),
                    train_loss_list=train_losses,
                    val_loss_list=val_losses,
                    global_steps_list=global_steps   
                )
            model.train()
              
save_metrics(
    p.join(exp_path, 'metrics.pt'),
    train_loss_list=train_losses,
    val_loss_list=val_losses,
    global_steps_list=global_steps
)

# save_params(
#     p.join(exp_path, 'params.pt'),
#     params={}
# )
"Done."

Evaluation.

In [ ]:
threshold = 0.5
y_pred = []
y_true = []
temp = []

model.eval()
with torch.no_grad():
    for test_ids, test_sentences, test_labels in loader_test:           
        test_labels = test_labels.float().to(device)
        output = model(test_sentences)

        temp.extend(output.tolist())
        output = (output > threshold).int()
        y_pred.extend(output.tolist())
        y_true.extend(test_labels.tolist())

print('Ranking:')
temp.sort(reverse=True)
print(temp[:10])

print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
# print(f'precision: {tp/(tp+fp)}')
# print(f'recall: {tp/(tp+fn)}')

print('Classification Report:')
print(classification_report(y_true, y_pred, digits=4)) # labels=[1,0], 

cm = confusion_matrix(y_true, y_pred)
ax = plt.subplot()
sns.heatmap(cm, annot=True, ax=ax, cmap='Blues', fmt="d")

ax.set_title('Confusion Matrix')

ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')

ax.xaxis.set_ticklabels(['worthy', 'unworthy'])
ax.yaxis.set_ticklabels(['worthy', 'unworthy'])